# Video Keypoint Detection

<table style="margin-left: 0px; margin-top: 20px; margin-bottom: 20px;">
<tr>
<td style="width:120px; padding-top: 10px; padding-bottom: 10px;"><img src="assets/Logo-TSP-IPP.png" style="height: 130px;"></td>
<td style="padding-left: 12px;">
<table style="width: 100%;">
<tr>
<th style="text-align: left; width: 80px;">File</th>
<td style="text-align: left;">OpenCV_VideoKeypointDetection.ipynb</td>
</tr>
<tr>
<th style="text-align: left;">Author</th>
<td style="text-align: left;">Nicolas ROUGON</td>
</tr>
<tr>
<th style="text-align: left;">Affiliation</th>
<td style="text-align: left;">Institut Polytechnique de Paris &nbsp;|&nbsp; Telecom SudParis &nbsp;|&nbsp; ARTEMIS Department</td>
</tr>
<tr>
<th style="text-align: left;">Date</th>
<td style="text-align: left;">July 31, 2022</td>
</tr>
<tr>
<th style="text-align: left;">Description</th>
<td style="text-align: left;">OpenCV sample routine &nbsp;>&nbsp; Differential (structure tensor-based) keypoint detection in video stream</td>
</tr>
</table>
</td>
</tr>
</table>
    
<b>OpenCV Documentation</b>
<div style="margin-top: 2px;">Corner detection</div>
<ul style="margin-top: 1px;">
<li><a href="https://docs.opencv.org/4.6.0/dd/d1a/group__imgproc__feature.html#gac1fc3598018010880e370e2f709b4345">cornerHarris</a> &nbsp;|&nbsp; Harris corner map</li>
<li><a href="https://docs.opencv.org/4.6.0/dd/d1a/group__imgproc__feature.html#ga3dbce297c1feb859ee36707e1003e0a">cornerMinEigenVal</a> &nbsp;|&nbsp; KLT corner map</li>
<li><a href="https://docs.opencv.org/4.6.0/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541">goodFeaturesToTrack</a> &nbsp;|&nbsp; KLT / Harris corner detector</li>
</ul>

<b>Tutorial</b>
<div style="margin-top: 2px;"><a href="https://docs.opencv.org/4.6.0/d9/d97/tutorial_table_of_content_features2d.html">feature2d</a> &nbsp;|&nbsp; 2D Features framework</div>
<ul style="margin-top: 1px;">
<li><a href="https://docs.opencv.org/4.6.0/d4/d7d/tutorial_harris_detector.html">Harris corner detector</a></li>
<li><a href="https://docs.opencv.org/4.6.0/d8/dd8/tutorial_good_features_to_track.html">KLT corner detector</a></li>
</ul>


In [5]:
import cv2 as cv
import numpy as np
import sys

sys.path.append("c:/Users/rougon/Notebooks")
from OpenCV_Image_Utilities import *       # Provides routine overlay_uchar_image()

##
## Global Variables
##

# - Camera video stream
isGrayCamera = False
frame_count = 0

# - Gaussian smoothing
trackbarLocalScale_name = "10*LScale"
trackbarLocalScale_max_value = 75           # Trackbar max value
trackbarLocalScale_min_value = 1            # Trackbar min value
trackbarLocalScale_value = 1                # Trackbar value
trackbarLocalScale_stepsize = 0.1           # Quantization step
local_scale_default_value = 1.0             # Gaussian kernel std deviation default value

# - Corner metric
trackbarKPDetector_name = "KLT|Harris"
trackbarKPDetector_max_value = 1
trackbarKPDetector_value = 1                # Trackbar value
useHarrisDetector= True                     # Keypoint detector
                                            # - Shi-Tomasi if "False"
                                            # - Harris if "True"
useHarrisDetector_default_value = False

# - Harris detector hyperparameter
trackbarHarrisParameter_name = "100*k"
trackbarHarrisParameter_max_value = 25      # Trackbar max value
trackbarHarrisParameter_min_value = 1       # Trackbar min value
trackbarHarrisParameter_value = 1           # Trackbar value
trackbarHarrisParameter_stepsize = 0.01     # Quantization step
HarrisParameter_default_value = 0.04

# - Integration scale (window half-size)
trackbarDetectorScale_name = "IScale"
trackbarDetectorScale_max_value = 10        # Trackbar max value
trackbarDetectorScale_min_value = 1         # Trackbar min value
trackbarDetectorScale_value = 1             # Trackbar value
trackbarDetectorScale_stepsize = 2          # Quantization step
DetectorScale_default_value = 3

# - Max # of detected keypoints
trackbarKPMaxCount_name = "Max #pts"
trackbarKPMaxCount_max_value = 1000         # Trackbar max value
trackbarKPMaxCount_min_value = 10           # Trackbar min value
trackbarKPMaxCount_value = 10               # Trackbar value
KPMaxCount_default_value = 500

# - Keypoint separation
trackbarKPMinDistance_name = "Min dist"
trackbarKPMinDistance_max_value = 128       # Trackbar max value
trackbarKPMinDistance_value = 1             # Trackbar value
KPMinDistance_default_value = 10.

# - Quality level
#   Corner metric upper threshold = fraction of maximal corner metric value
trackbarQualityLevel_name = "Quality level"
trackbarQualityLevel_max_value = 100        # Trackbar max value
trackbarQualityLevel_min_value = 1          # Trackbar min value
trackbarQualityLevel_value = 1              # Trackbar value
trackbarQualityLevel_stepsize = 0.01        # Quantization step
qualityLevel_default_value = 0.01

# Graphics | Detected keypoint overlay
KPColor = (0,255,0)
KPRadius = 2
KPThickness = -1

##
## IMAGE PIPELINE COMPONENTS 
##

#
# Graphical User Interface 
#
def create_GUI():
                                  # Windows
    global window_settings_name   # GUI
    global window_out_name        # Displays keypoints onto original image
    
    global window_name_prefix
    
    window_name_prefix = 'OpenCV Demo | Keypoint detection > '
    
    # Create windows
    # - for keypoint map overlay
    window_out_name = window_name_prefix + 'Detected keypoints'
    cv.namedWindow(window_out_name, cv.WINDOW_AUTOSIZE)

    # - for hyperparameter settings
    window_settings_name = window_name_prefix + 'Settings'
    cv.namedWindow(window_settings_name, cv.WINDOW_AUTOSIZE)
    
    # Create trackbars
    # - for Gaussian kernel std deviation
    cv.createTrackbar(trackbarLocalScale_name, window_settings_name, 0,
                      trackbarLocalScale_max_value, process_display_callback)
    cv.setTrackbarMin(trackbarLocalScale_name, window_settings_name, 
                      trackbarLocalScale_min_value)
    
    # - for KP detector
    cv.createTrackbar(trackbarKPDetector_name, window_settings_name, 0,
                      trackbarKPDetector_max_value, process_display_callback)
    
    # - for Harris detector hyperparameter
    cv.createTrackbar(trackbarHarrisParameter_name, window_settings_name, 0,
                      trackbarHarrisParameter_max_value, process_display_callback)
    
    cv.setTrackbarMin(trackbarHarrisParameter_name, window_settings_name,
                    trackbarHarrisParameter_min_value)
    
    # - Detector (half) scale
    cv.createTrackbar(trackbarDetectorScale_name, window_settings_name, 0,
                      trackbarDetectorScale_max_value, process_display_callback)

    cv.setTrackbarMin(trackbarDetectorScale_name, window_settings_name,
                      trackbarDetectorScale_min_value)

    # - Maximum # of detections
    cv.createTrackbar(trackbarKPMaxCount_name, window_settings_name, 0,
                      trackbarKPMaxCount_max_value, process_display_callback)
  
    cv.setTrackbarMin(trackbarKPMaxCount_name, window_settings_name,
                      trackbarKPMaxCount_min_value)
  
    # - Keypoint minimum distance
    cv.createTrackbar(trackbarKPMinDistance_name, window_settings_name, 0,
                      trackbarKPMinDistance_max_value, process_display_callback)

    # - Quality level, defined as fraction of the largest minimal eigenvalue of structure tensor over the image domain
    cv.createTrackbar(trackbarQualityLevel_name, window_settings_name, 0,
                      trackbarQualityLevel_max_value, process_display_callback)

    cv.setTrackbarMin(trackbarQualityLevel_name, window_settings_name,
                      trackbarQualityLevel_min_value)

    # Set trackbar default positions
    # - for Gaussian kernel std deviation
    trackbarLocalScale_value = (int)(local_scale_default_value / trackbarLocalScale_stepsize)
    cv.setTrackbarPos(trackbarLocalScale_name, window_settings_name,
                      trackbarLocalScale_value)

    # - for KP detector
    if useHarrisDetector_default_value == True:
        trackbarKPDetector_value = 1
    else:
        trackbarKPDetector_value = 0
    cv.setTrackbarPos(trackbarKPDetector_name, window_settings_name,
                      trackbarKPDetector_value)

    # - for Harris detector hyperparameter
    trackbarHarrisParameter_value = (int)(HarrisParameter_default_value / trackbarHarrisParameter_stepsize)
    cv.setTrackbarPos(trackbarHarrisParameter_name, window_settings_name,
                      trackbarHarrisParameter_value)
    
    # - for Detector (half) scale  
    trackbarDetectorScale_value = (int)((DetectorScale_default_value - 1) / trackbarDetectorScale_stepsize)
    cv.setTrackbarPos(trackbarDetectorScale_name, window_settings_name,
                      trackbarDetectorScale_value)

    # - for Maximum # of detections
    trackbarKPMaxCount_value = KPMaxCount_default_value
    cv.setTrackbarPos(trackbarKPMaxCount_name, window_settings_name,
                      trackbarKPMaxCount_value)

    # - for Keypoint minimum distance
    trackbarKPMinDistance_value = (int)(KPMinDistance_default_value)
    cv.setTrackbarPos(trackbarKPMinDistance_name, window_settings_name,
                      trackbarKPMinDistance_value)

    # - for Quality level
    trackbarQualityLevel_value = (int)(qualityLevel_default_value / trackbarQualityLevel_stepsize)
    cv.setTrackbarPos(trackbarQualityLevel_name, window_settings_name,
                      trackbarQualityLevel_value)

#
# Video stream initialization
# 
def initialize_stream():
    global cap
    global frame
    global isGrayCamera
    
    # Open default camera
    cap = cv.VideoCapture(0)

    # Check if camera opening is successful
    if not cap.isOpened():
        sys.exit("! Cannot open default camera")

    # Get camera properties and initialize various variables
    # - Check for color camera
    status, frame = cap.read()       # Get frame
    if not status:
        sys.exit("! Cannot grab frame from default camera")
    if frame.ndim == 1:
        isGrayCamera = True
    else:
        isGrayCamera = False

        
#
# Input & Preprocessing
#
def grab_preprocess():
    global frame, the_frame
    
    # Get a new frame from camera
    status, frame = cap.read()
    ++frame_count
    
    # Convert image to graylevel if appropriate
    if isGrayCamera:
        the_frame = frame
    else:
        the_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
#
# Trackbar callback
#
def process_display_callback(value):
    global local_scale           # Gaussian kernel std deviation
    global HarrisParameter       # Harris parameter
    global DetectorScale         # Integration scale
    global KPMaxCount            # Max # of detected KP
    global KPMinDistance         # KP minimal distance
    global qualityLevel          # Quality level
    global useHarrisDetector     # Keypoint detector
                                 # - Shi-Tomasi if "False"
                                 # - Harris if "True"

    # Get trackbars positions
    trackbarLocalScale_value = cv.getTrackbarPos(trackbarLocalScale_name, window_settings_name)
    trackbarKPDetector_value = cv.getTrackbarPos(trackbarKPDetector_name, window_settings_name)
    trackbarDetectorScale_value = cv.getTrackbarPos(trackbarDetectorScale_name, window_settings_name)
    trackbarHarrisParameter_value = cv.getTrackbarPos(trackbarHarrisParameter_name, window_settings_name)
    trackbarKPMaxCount_value = cv.getTrackbarPos(trackbarKPMaxCount_name, window_settings_name)
    trackbarQualityLevel_value = cv.getTrackbarPos(trackbarQualityLevel_name, window_settings_name)
    trackbarKPMinDistance_value = cv.getTrackbarPos(trackbarKPMinDistance_name, window_settings_name)
    
    # Set hyperparameter values from trackbars
    local_scale = (float)(trackbarLocalScale_value)*trackbarLocalScale_stepsize
    DetectorScale = trackbarDetectorScale_stepsize*trackbarDetectorScale_value + 1

    if trackbarKPDetector_value == 1:
        useHarrisDetector = True
    else:
        useHarrisDetector = False

    HarrisParameter = (float)(trackbarHarrisParameter_value)*trackbarHarrisParameter_stepsize
    qualityLevel = (float)(trackbarQualityLevel_value)*trackbarQualityLevel_stepsize    
    KPMinDistance = (float)(trackbarKPMinDistance_value)
    KPMaxCount = trackbarKPMaxCount_value
    
    if verbosity == True:
        if useHarrisDetector == True:
            detector_params = " (k = {})"
            detector = "Harris" + detector_params.format(HarrisParameter)
        else:
            detector = "Shi-Tomasi"
        print("> Detector: ", detector, " | Local scale: ", local_scale, " | Integration scale:", DetectorScale, " | Quality level: ", qualityLevel, " | Max #points: ", KPMaxCount, " | Min distance: ", KPMinDistance)
    
    # Processing & Visualization
    if (qualityLevel > 0) and (KPMinDistance >= 0) and (KPMaxCount >= 0):
        process_display()

#
# Processing & Visualization
#
def process_display():    
    # Preprocessing > Gaussian filtering
    # - Kernel size is set to 0, and is automatically estimated from sigma
    # - Last argument "BorderType" of GaussianBlur() is omitted,
    #   so that default boundary conditions (BORDER_DEFAULT) are used
    if local_scale > 0:
        image_out = cv.GaussianBlur(the_frame, (0,0), local_scale)
    else:
        image_out = the_frame.copy()
  
    # Differential corner detectionµµ
    keypoints = cv.goodFeaturesToTrack(image_out, KPMaxCount, qualityLevel, KPMinDistance, \
                                       None, None, DetectorScale, useHarrisDetector, HarrisParameter)
            
    # Overlay keypoint map onto original image
    image_out = frame.copy()
    for point in np.int0(keypoints):
        image_out = cv.circle(image_out, point.ravel().tolist(), KPRadius, KPColor, KPThickness, cv.LINE_AA)

    # Display detection statistics
    nb_keypoints = np.size(keypoints,0)
    window_title = window_name_prefix + " Frame  #{}: {} points".format(frame_count, nb_keypoints)
    cv.setWindowTitle(window_out_name, window_title)
            
    # Display resulting images
    cv.imshow(window_out_name, image_out)
  
    # Clear detected points for next round
    keypoints = [ None ]
    

##
## MAIN ROUTINE
##

def application():
    # Open camera & Get its features
    initialize_stream()
    
    # Grab & preprocess frame
    grab_preprocess()

    # GUI creation
    create_GUI()

    # Processing & Visualization
    # - Invoke callback routine to initialize and process
    process_display_callback(trackbarLocalScale_value)

    # Process video stream
    while True:
        # Grab & preprocess frame
        grab_preprocess()
        
        # Processing & Visualization
        process_display()
        
        # Listen to next event
        if cv.waitKey(5) >= 0:
            break

    # Release camera
    cap.release()
    
    # Destroy windows
    cv.destroyAllWindows()

<b>Run the application</b>

In [6]:
# - Silent / Verbose mode
verbosity = False

# Run application
application()